# Keyboard BCI

The name "brain-computer interface" suggests that you're using your brain to control a computer. In this notebook, we build a BCI whose `action` is to send keystrokes to the computer.

You can probably think of a number of different applications for something like this. One example would be to use your BCI as a game controller where a certain kind of brain signal is a trigger for a keystroke corresponding to a certain in-game action.

In [1]:
# importing the `generic_BCI` class
from neurol.BCI import generic_BCI

For simplicity, let's have our trigger be a blink for this BCI. `neurol` has a blink classifier that we can use. 

In [2]:
from neurol.models import model_tools

blink_model = model_tools.get_model('blink_balanced')

This particular model is trained on epochs of 125 samples of two channels (AF7, AF8). So it expects epochs of shape `[125, 2]`, and the predictor classifies on inputs of shape `[n_epochs, 125, 2]`. 

Let's use the model on some dummy input to see what it's output looks like.

In [3]:
import numpy as np

input_ = np.random.random([10, 125, 2]) # 10 random epochs of two channels

pred = blink_model.predict(input_)
print(pred)

[[1.0000000e+00 3.1113129e-10]
 [1.0000000e+00 4.0340478e-10]
 [1.0000000e+00 7.2555750e-10]
 [1.0000000e+00 2.4457986e-10]
 [1.0000000e+00 4.4044490e-10]
 [1.0000000e+00 2.6594454e-10]
 [1.0000000e+00 6.2772970e-10]
 [1.0000000e+00 5.8554828e-10]
 [1.0000000e+00 3.4997380e-10]
 [1.0000000e+00 8.1923252e-10]]


The output is a softmax prediction (first entry for the non-blink classification and second for blink)

Let's use it to build our BCI's classifier

In [4]:
def blink_clf(buffer, clb_info):
    # get the latest 125 samples from the channels we want
    input_ = buffer[-125:, 1:3] 

    # expand dims so that input is of shape [1, 125, 2] 
    input_ = np.expand_dims(input_, axis=0) 

    # run model to make prediction
    pred = blink_model.predict(input_)

    # get index of higher probability prediction
    pred_idx = np.argmax(pred) 

    # determin whether prediction is a blink
    is_blink = bool(pred_idx)

    return is_blink

Now we can define the action we want our BCI to perform based on the results of the classifier.

Let's say we want our BCI to send a spacebar keystroke every time it detects a blink. 

To do this, we'll use the `pynput` package.

In [5]:
from pynput.keyboard import Key, Controller

keyboard = Controller()


def key_action(is_blink):
    # press spacebar if classifier thinks a blink occured
    if is_blink:
        keyboard.press(Key.space)
        keyboard.release(Key.space)

That's all we need. We're going to keep it simple and not bother with a transformer or calibrator.

We can now define the BCI!

In [6]:
keyboard_bci = generic_BCI(blink_clf, transformer=None, action=key_action, calibrator=None)

Finally, we'll set up our stream.

In [7]:
from neurol.connect_device import get_lsl_EEG_inlets
from neurol import streams

inlet = get_lsl_EEG_inlets()[0] # gets first inlet, assuming only one EEG streaming device is connected

# we ask the stream object to manage a buffer of 1024 samples from the inlet
stream = streams.lsl_stream(inlet, buffer_length=1024)

Okay! We can test it out now.

In [8]:
try:
    keyboard_bci.run(stream)

except KeyboardInterrupt:
    stream.close()
    
    print('\n')
    print('QUIT BCI')



QUIT BCI
